In [1]:
import torch
import torch.distributions as dist

In [68]:
import hydra

ModuleNotFoundError: No module named 'hydra'

In [2]:
def P_C(c : torch.Tensor) -> torch.Tensor:
    probs = torch.Tensor([0.5, 0.5])
    d = dist.Categorical(probs)
    return torch.exp(d.log_prob(c))

def P_S_given_C(s : torch.Tensor, c : torch.Tensor) -> torch.Tensor:
    probs = torch.tensor([[0.5, 0.5], [0.9,0.1]])
    d = dist.Categorical(probs)
    
    return torch.exp(d.log_prob(s))[c.detach()]

def P_R_given_C(r : torch.Tensor, c : torch.Tensor) -> torch.Tensor:
    probs = torch.tensor([[0.8, 0.2], [0.2, 0.8]])
    d = dist.Categorical(probs)
    return torch.exp(d.log_prob(r)[c.detach()])

def P_W_given_S_R(w : torch.Tensor, s:torch.Tensor, r : torch.Tensor) -> torch.Tensor:
    probs = torch.tensor([ [[1.0, 0.1], [0.1, 0.01]], [[0.0, 0.9], [0.9, 0.99]] ])
    return probs[w.detach(), s.detach(), r.detach()]

In [3]:
#compute joint
p = torch.zeros((2,2,2,2)) #c, s, r, w
for c in range(2):
    for s in range(2):
        for r in range(2):
            for w in range(2):
                p[c, s, r, w] = P_C(torch.tensor(c)) * \
                P_R_given_C(torch.tensor(r), torch.tensor(c)) * \
                P_S_given_C(torch.tensor(s), torch.tensor(c)) * \
                P_W_given_S_R(torch.tensor(w), torch.tensor(s), torch.tensor(r))
                
                

In [4]:
#compute P_C=1_given_W=1 
(p[:,:,:,1].sum(axis=[1,2])/p[:,:,:,1].sum(axis=[0,1,2]))[1]

tensor(0.5758)

In [73]:
#ancestral and rejection sampling

num_samples = 10000
samples = torch.zeros(num_samples) #P(C, W)
rejections = 0 #to compute P(W)
i = 0

# NOTE: Fill this in

while i < num_samples:
            
    c = dist.Categorical(torch.tensor([0.5,0.5])).sample().item() #p(c)

    if c==0:
        r = dist.Categorical(torch.tensor([0.8, 0.2])).sample().item() #p(r|c=0)
        s = dist.Categorical(torch.tensor([0.5, 0.5])).sample().item() #p(s|c=0)
        
    elif c==1:
        r = dist.Categorical(torch.tensor([0.2, 0.8])).sample().item() #p(r|c=1)
        s = dist.Categorical(torch.tensor([0.9, 0.1])).sample().item() #p(s|c=1)


    if r+s==1:
        w = dist.Categorical(torch.tensor([0.1, 0.9])).sample().item() #p(w|r+s=1)

    elif r+s==2:
        w = dist.Categorical(torch.tensor([0.01, 0.99])).sample().item() #p(w|r+s=2)

    else:
        w=0
    
    if w==0:
        rejections+=1
        continue
    
    if w==1 & c==1:
        samples[i]=1
    
    i+=1

In [74]:
samples.mean()

tensor(0.5736)

In [179]:
rejections

5355

In [6]:
p_R_given_C_S_W = p/p.sum(axis=2, keepdims=True)
p_S_given_C_R_W = p/p.sum(axis=1, keepdims=True)

p_C_S_R = torch.zeros((2,2,2)) #c,s,r
for c in range(2):
    for s in range(2):
        for r in range(2):
            p_C_S_R[c, s, r] = P_C(torch.tensor(c).int())*P_S_given_C(torch.tensor(s).int(), torch.tensor(c).int())*P_R_given_C(torch.tensor(r).int(), torch.tensor(c).int())

#then create the conditional distribution:
p_C_given_S_R = p_C_S_R[:,:,:]/p_C_S_R[:,:,:].sum(axis=(0),keepdims=True)

In [69]:
##Gibbs sampling
num_samples = 11000
samples = torch.zeros(num_samples)
state = torch.zeros(4) #c, s, r, w
state[3] = 1 #set w = True

# NOTE: Fill this in
i=0
while i < num_samples:
    c,s,r,w = state.int().numpy()
    
    r = dist.Categorical(p_R_given_C_S_W[c,s,:,w]).sample().item()
    s = dist.Categorical(p_S_given_C_R_W[c,:,r,w]).sample().item()
    c = dist.Categorical(p_C_given_S_R[:,s,r]).sample().item()
    
    if c==1:
        samples[i]=1
    
    state[0] = c
    state[1] = s
    state[2] = r

    i+=1
    


In [70]:
samples.mean()

tensor(0.5814)

In [67]:
p[c,s,r,w].item()

0.32399997115135193